In [346]:
def evclid (a, b):
    if(b == 0):
        return (1, 0, a)
    v, u, d = evclid(b, a%b)
    return (u, v - (a//b)*u, d)
def obern (a, n):
    u, v, d = evclid(a, n)
    if d != 1: 
        print("No reverse")
        return 0
    else: return u
def rozv(a, b, n):
    x = []
    u, v, d = evclid(a, n)
    if d == 1:
        x.append((obern(a, n) * b)%n)
    else: 
        if b%d == 0:
            x0 = rozv(a/d, b/d, n/d)
            for i in range(d):
                x.append(int(x0[0] + i*n/d))
    return x

In [347]:
import codecs
f = codecs.open('10.txt', 'r', 'utf-8')
data = f.read()

In [348]:
import re
def filtr(data):
    reg = re.compile('[^а-яА-ЯёЁ]')
    data = reg.sub('', data)
    data = data.lower()
    data = data.replace('ъ', 'ь')
    data = data.replace('ё', 'е')
    return data

In [349]:
import numpy as np
def bi_freq(data):
    bi = np.zeros((31, 31))
    try:
        for n in range(0, len(data) - 1, 2):
            if (ord(data[n]) > 1097) and (ord(data[n+1]) > 1097):                
                bi[ord(data[n]) - 1073][ord(data[n+1]) - 1073] += 1
            elif ord(data[n]) > 1097:
                bi[ord(data[n]) - 1073][ord(data[n+1]) - 1072] += 1
            elif ord(data[n+1]) > 1097:
                bi[ord(data[n]) - 1072][ord(data[n+1]) - 1073] += 1
            else:
                bi[ord(data[n]) - 1072][ord(data[n+1]) - 1072] += 1
    except:
        print('break')
        print(n)
    bi = np.divide(bi, len(data)//2)
    return bi

In [350]:
data = filtr(data)
freq = bi_freq(data)

In [351]:
alf = ''
a = ord('а')
alf = alf.join([chr(i) for i in range(a, a+26)] + [chr(i) for i in range(a+27, a+32)])
alf.replace('ы', '-').replace('ь', 'ы').replace('-', 'ь')
list_alf = list(alf)
def max5(arr_freq):
    most_freq = []
    frequency = []
    frequency = arr_freq.copy()
    for i in range(5):
        x = np.argmax(frequency) // frequency.shape[0]
        y = np.argmax(frequency) % frequency.shape[1]
        most_freq.append(list_alf[x] + list_alf[y])
        frequency[x][y] = 0
    return most_freq

In [352]:
most_enc = max5(freq)

In [353]:
most_enc

['сг', 'жэ', 'ям', 'нг', 'тм']

In [355]:
def numb(s):
    num = 0
    if ord(s) < 1098:
        num = ord(s) - 1072
    else: num = ord(s) - 1073
    return num
def bi_num(bi):
    num = 0
    num = numb(bi[0])*31 + numb(bi[1])
    return num
def find_bi(num):
    bi = ""
    bi += list_alf[num//31]
    bi += list_alf[num%31]
    return bi

In [370]:
def decrypt(data, a, b, n):
    decrypted_d = ""
    for i in range(0, len(data), 2):
        x = rozv(a, bi_num(data[i] + data[i+1]) - b, n**2)
        if x:
            for j in x:
                decrypted_d += find_bi(j)
    return decrypted_d
def meaningful (data):
    d = {}
    d = dict.fromkeys(data, 0)
    for symb in data:
        d[symb] += 1
    for key in d:
        d[key] /= len(data)
    if d['а'] < 0.05:
        print('{}:{}'.format('a', d['а']))
        return 0
    if d['е'] < 0.05:
        print('{}:{}'.format('e', d['е']))
        return 0
    if d['о'] < 0.07:
        print('{}:{}'.format('o', d['о']))     
        return 0
    if d['ф'] > 0.03 or d['щ'] > 0.03 or d['ь'] > 0.05:
        print("freq ф or щ or ь more then have to be")
        print(d['ф'], d['щ'], d['ь'])
        return 0
    return 1

In [357]:
most_dec = ['ст', 'но', 'то', 'на', 'ен']

In [358]:
import itertools
most_enc_2 = []
most_dec_2 = []
for i in itertools.product(most_enc, most_enc):
    if i[0] != i[1]:
        most_enc_2.append(i)
for i in itertools.product(most_dec, most_dec):
    if i[0] != i[1]:
        most_dec_2.append(i)

In [359]:
list_x = []
for i in itertools.product(most_enc_2, most_dec_2):
    a = rozv(bi_num(i[1][0]) - bi_num(i[1][1]), bi_num(i[0][0]) - bi_num(i[0][1]), 31**2)
    if a: 
        for j in a:
            b = rozv(1, bi_num(i[0][0]) - j*bi_num(i[1][0]), 31**2)
            if b:
                for k in b:
                    list_x.append([j, k])

In [360]:
res=[]
for i in list_x:
    if i not in res:
        res.append(i)

In [361]:
len(res)

285

In [371]:
suitable=[]
for i in res:
    plain = decrypt(data, i[0], i[1], 31)
    print(i)
    if meaningful(plain):
        suitable.append(i)
        print("Can be our key")

[513, 596]
a:0.026281208935611037
[451, 751]
a:0.029697766097240473
[300, 400]
Can be our key
[243, 713]
a:0.017739816031537452
[448, 148]
a:0.0328515111695138
[709, 865]
e:0.03495400788436268
[750, 105]
e:0.04756898817345598
[510, 954]
a:0.03994743758212878
[616, 865]
e:0.040867279894875166
[781, 663]
e:0.04783180026281209
[661, 344]
a:0.042706964520367936
[252, 840]
e:0.040473061760841
[345, 840]
a:0.04770039421813403
[672, 716]
e:0.030617608409986858
[718, 31]
a:0.01419185282522996
[211, 639]
a:0.04204993429697766
[180, 81]
a:0.04559789750328515
[289, 28]
e:0.02706964520367937
[207, 152]
Can be our key
[728, 663]
e:0.013797634691195795
[705, 705]
a:0.025886990801576873
[754, 359]
a:0.02483574244415243
[657, 446]
e:0.040735873850197106
[859, 780]
a:0.025492772667542705
[661, 111]
a:0.021813403416557162
[316, 446]
e:0.03771353482260184
[332, 904]
a:0.014848883048620237
[233, 809]
a:0.048751642575558474
[304, 65]
a:0.04625492772667543
[645, 65]
e:0.020105124835742443
[170, 251]
e:0.029

[734, 90]
Can be our key
[765, 617]
a:0.04993429697766097
[796, 183]
o:0.06793692509855453
[827, 710]
Can be our key
[858, 276]
Can be our key
[889, 803]
o:0.0671484888304862
[920, 369]
Can be our key
[951, 896]
o:0.06254927726675427
[620, 942]
e:0.0277569392348087
[248, 942]
e:0.02194298574643661
[341, 942]
e:0.016129032258064516
[310, 756]
a:0.023105776444111027
[589, 12]
a:0.02194298574643661
[620, 570]
o:0.048687171792948236
[362, 121]
Can be our key
[635, 291]
e:0.01695137976346912
[178, 457]
a:0.03521681997371879
[692, 142]
a:0.025755584756898816
[812, 74]
e:0.03823915900131406
[146, 67]
a:0.011432325886990802
[599, 855]
a:0.016557161629434953
[471, 74]
e:0.03442838370565046
[580, 191]
a:0.022733245729303547
[326, 685]
e:0.03521681997371879
[149, 902]
a:0.04533508541392904
[490, 902]
a:0.04993429697766097
[77, 127]
e:0.034034165571616294
[783, 519]
a:0.03574244415243101
[815, 909]
a:0.04441524310118265
[381, 785]
e:0.04822601839684625
[884, 849]
e:0.028515111695137978


In [372]:
len(suitable)

21

In [365]:
for i in suitable:
    plain = decrypt(data, i[0], i[1], 31)
    print(i)
    print(plain)

[300, 400]
поздновечеромнаверандесиделколяичтотописглвтемнотебумагуитутолкомнелызябьллразалядетывремяотвреминиорвосклицглагаиливщтотожезначитемувголовуприходилоещечтонибудыподходящеедляегоспискапотомдверычутыстукнулаточновсеткуотмоскитовудариласыночнаябабочкалинашепнулауфманонаселарядомснимнакачеливоднойночнойсорочкинетониныкаякаксемнадцатилетняядевочкакоторуюещенелюбятинетолставкакпятидесятилетняяжеуэенакотлруюужинелюбятноскоаднаяикрепкаяиминнотакавкакнадотаковьжеуэеньвовсякомвозрастееслионилюбимьонабьлаудивителынаяеетелокакиегособствинноевсегдадумглозанеетолыкоподругомуоновьнашивалодетейиливходиловпередилеовкаждуюкомнатучтобьнеуловимоизминитытамсамьйвоздухподстатынастроениюмужаказглосыонаникогданезадумьваетсянадолгомьслытотчаспередавгласыотееголовьплечампглчцамипретвлряласывдействиетакнезаметноиестествинночтолеонесмогбьдаенехотелизобразитыэтокакимилибочертежамиэтамашенасказаоаонанаконецненужнаонанамдаотозвглсяонноеногданужнопозаботитысяиодругихявотвседумаючтотудавставитыкинокантеньра

[52, 400]
кокдиоюеуелоонсвсрвнаечиаегкялеихтмтвпнсулатомиооечумащужтстялеоонхлизчбмлчрнзрлвдгтмвмеуямторомкнговвусылйцулкгжисинщноновейнщчжтомевюожошуьррхсдшлкехехтрнббцдопсдроляхеидтяпгусхийкупмтчмхвсрфчстасйумндлютичиозсгтбумтзойкжтавцдмршлесэничначбшбичкасинафесндлцуюмвнрнесхлмрвдчмунсмнакауесиэожннйиощннймолопккнгтрнкнткзякапсомнаецютшлгтфявдцвичкаеонозущехейещюияшийенорстащкчкцяшиаешяшизефнжявейэзнчкмтчрвюижкнхлцбэтиойкоажнзяакмезкзясмкниотакащкчкнаяотаеояьвейэзннвавшяеоювьзраптбевлгоуищюзийьрншбмлцукиииоепынаыегтхлжкчкдеюоможсдвкниоцвнеждгдэмульзвнбенопыеокоррюгчмооиояьнаяиважоаеоеписикхсдшлавлемекизеавкайдвюеоонютнчноюьйедлавсмфирмкнжтштймсайьывьзыуйпсдптютэнесяркеуизмижчкнзулусхонауиеождвнтзгдэмнвьечсбнгдялюойьвлштмтчаепсридсвулесхооепгялавппзечаапулшцймьпмедвчрплесмваеосдвдетамнтзймгтиодептксдвкниохтялаойеымшгюьдазннхмтхлхззбраницыытжкчксмшлббичсроежатиытймтшзнесказаоарнвнчкрнжцйедуиныонанапдыоношвулшярниозншгдадуинвпьзшбмтжтасеисдзуйиэяэодвнеыумаччнойудазстаиицырииокалтзниргд

[83, 555]
пооднояефероцньврркнбепибефкнлэиотетдпйсвлштщмнопенуэапуятктнлкоцндлгзебылцризялждьтцвнезяетнрщмтниомвпсйлхцвлсгюийирщтотогеснгчятщмпвголооуыраххдзллецеотшнобыдрпхдхояяцемджяцгпсниыкхпетгмавррючктьсяуфнтлцтсчногсьтцуетмоыкятквыдлрзлбсенсчюаебжбсчыайиюахещнтлмуймкншнбсдллрждгмьнэмюаыафейивоонвйновнвйсоробктньтшнтндкыяыалсщмюасццтзльтияждбвсчыакотоэуыецекерюьярикетомсгалкйккярибемярииеэнщягегэпнйкетцршюжжтндлдбхтноыкяаоныяскнешкыяэмтнногаыалкйкюадогакоььгегэпнчвквмякоивцзбаитверлиолирюяижьшнжбылмувиаипепыюаьеьтдлчкйкеегосовсовтннобвоекдздимвлцзкнветопыкопопреггмуоноььюациталобепезийищххдзлквменевииеквыандшюкоцнцтцчтоыькетлквэммиьмтнятстфмважьевцзрулпхдитцтенбсюрлелитмжжйкизвлпсыоюаликокдкннзздимчвэеусйнзднлгожьрлстетиазпррмдьввлбсыопецгнлквспиеиавпвлецфмюпнеовцрюлбсцвбексовеегафннзфмьтноееитжсовтнноотнлеокежмагыьфапнэхетдлрзфббаеицыутчкйкэмзлобсчррпецакиутфмшшпнбсыачаяашнкнйкшнтцкещурнаоюаюаудаотогввлмяшннопнагфащурндпцзжбетятьсэихдэубирявоовоеруэабчтояуфагсгааицыииноыадтпнзрзд

[207, 741]
покднооедерогнрвчрцнрещирежктлзиэтутчпдсзлзттмнояепукасунтшттлкогнйлщзжбалюряздлвдктлвэебяутфртмяниобвксолтцзлщгииуиищтототеюнучнттмдвголоруврйхсдмлыежеэтенпбцддпсдхошяжеидаяягксчимкйпутьмфвчрочштцсбубнчлетвчноюсктшуутмомкнтявцдтрмльсснвчлажбзбвчиауилаеежнчлоувмцненьсйлтрвдьминхмлаиадеуивоынайноонайсороткянктенянхкфяиажстмлаоцетмлктвявдхввчиакотояукежещецюхяыищетозсраэкыкдяыирежяыичейнуятехэьныкутюряюйжянйлебдтномкмаынфягкэеккфяхмяннораиаэкыкладоракоыьтехэьнмвявжякоэвнзоацтсехлиохицюйиеьензбалоумикияеыылалектйлйкыкфегосоэсгвяннохвюеждгдбмзлнзцнсетоыыкопоцрнгьмуоноыьлабиаалореяесиуигхсдмлявьеэемичеявиайдяюкогнетзчтощьщечлявхмцифмяннтатнмпаеьщвнзтуяпсдцтетсньсерыехилмйжыкяззлксыолахикождцндзгдбммвмеосхнгдтлгоеьхлатутхаыпчридрвзльсыояеягтлявепчехахпзлвцнмспэегвюргльслвреесгвферабндзнмктнофецтбсгвянноэттлеощеямигщьваьнжхутйлззхбоапивывтйкыкхммлпбвччряедафивтнмэшьньсиаеамаенцныкенпцщеьуэнаолалапдаоточвзлжяенноьнигваьуэнчпнззбутнтцсзисдяуликявогвюетукасчтобуваюсракивытиноиаттьнорг

[827, 710]
похдноьесеробнчвьрфнюениюеюкмлыиетьтщпксблптомномещунаьухтбтмлкобнгллзлбщлвррзюлндтттвкевяьтшромэниоиврсилгцблггьизиящтотоаеьнжчхтомлвголоыужрчхэджлиеуеетгнфбвджпэдхощяуеудбяигрслидклпьтчмьвьрбчбтэслуянслнтфчнодсттгуьтмодкхтжввдцржлвспнфчоалбмбфчлазиоатеднслшуюмфнгнвсглцрндчмжнсмоаласезивошнхйномнхйсороккэнттгнэннкхяламсомоаяцнтжлттгяндэвфчлакотойучеуезехюцяоизетонсуафкткеяоиюезяоиеезнфяаетэщнткьтврююшжэнглкбмтнодкпашнхяыккевкхясмэнноуалафкткоадоуакохьаетэщнувжвзякодвязсаятяеплиойихюэиаьгнмбщлшуаиюимеуыоашеттглбктквегосогсквэнноэвлесдодэмблязфняетоуыкопоырцгчмуонохьоафигалоюемееизисхэджлжвйекеаиеежвлафдююкобннтщчтофьзеслжвсмкирмэнхтитймтааьбвязэубпэдятнтпнвсйриейизмшжткрзблрсыооайикосдфнхзодэмувщефсунодмлгоаьплитьтшаэпьрудчвблвсыомеигмлжвзпеешачпблтцймупкеквврэлвствюелскввеуаянхзймттновеятзсквэнноетмлеозеымсгфьеащнфхьтглшзыбсагиыыктбктксмжлфбфчьрмезаииктймсшщнвслаиапагнфнткгнацзежуынаооаоаыдаотоявблзягннощнсгеажуынщпязмбьтхтэсыиэдйуяилявоквлеэунадчтолуеадсуаюиыыжинолаытщнтро

[362, 121]
фокдтояефехочнпвюрлнбеьибейкжлйидтытнпысылотцмтопезуиайуфтатжлпочнэллзфбулдррзчлвдстквнеояытырцмунноавбсвльцылйгкихигщчочогетннчфтцмгвиороиуирвхсдаллецедтщнюбцдщпсдыожяцеидняогбсщипкяпытамувюричатнсшухнллмтьчтофсструытсопкфтювцдшралтсжньчйафбхбьчжахийахеынллжужмлнщнтсэлшрвдамэнщмйажафехизопнтйтогнтйцохохкунстщнуншквяжаэсцмйачцмталстпявдфвьчжапочоцуыецекеуюгяэикечоюсоаакюксяэибеуяэииеюнбягевэрнюкытдрьюкжунэлублттопккапнвяжкненквящмунтооажаакюкйайооапоцьгевэрнлвювуяпоьвязмаютвейлночиуюлибьщнхбулжуоимипежыйаьестэлмкюкееиоцоусввунтофвоеждгдемылязлнвечожыпофоэрэгамшотоцьйагиюаробепеуихиьхсдалювменеоииеювжайдьюпочнмтбччохькеллювщмшишмунфтзтсмнабьшвязкуфпсдютмтжнтслрлечипмкжюкрзылбсаойачипождлнхзгдемлвэеескнгджлиобьйлзтытуарпюридпвылтсаопеогжлювыпиеуалпыллцсмзпневвдрцлтсквбеьсввееоахнхзсмсттоееютчсввунтодтжлкокегмчгхьаарняхытэлшздбмасиныйтмкюкщмалюбьчюрпевацийтсмзшрнтсжагакащнлнюкщншцкеууснеойайапдеочоцвылуящнторнчгаауусннпязхбытфтнсйисдцуничязоввоекуиалччошуаафсоаминыфитожащтрнфрг

In [368]:
plaintext = decrypt(data, suitable[0][0], suitable[0][1], 31).replace('ы', '-').replace('ь', 'ы').replace('-', 'ь')

In [369]:
plaintext

'поздновечеромнаверандесиделколяичтотописглвтемнотебумагуитутолкомнельзябыллразалядетьвремяотвреминиорвосклицглагаиливщтотожезначитемувголовуприходилоещечтонибудьподходящеедляегоспискапотомдверьчутьстукнулаточновсеткуотмоскитовудариласьночнаябабочкалинашепнулауфманонаселарядомснимнакачеливоднойночнойсорочкинетонинькаякаксемнадцатилетняядевочкакоторуюещенелюбятинетолставкакпятидесятилетняяжеуэенакотлруюужинелюбятноскоаднаяикрепкаяиминнотакавкакнадотаковыжеуэенывовсякомвозрастееслионилюбимыонабылаудивительнаяеетелокакиегособствинноевсегдадумглозанеетолькоподругомуоновынашивалодетейиливходиловпередилеовкаждуюкомнатучтобынеуловимоизминитьтамсамыйвоздухподстатьнастроениюмужаказглосьонаникогданезадумываетсянадолгомысльтотчаспередавгласьотееголовыплечампглчцамипретвлряласьвдействиетакнезаметноиестествинночтолеонесмогбыдаенехотелизобразитьэтокакимилибочертежамиэтамашенасказаоаонанаконецненужнаонанамдаотозвглсяонноеногданужнопозаботитьсяиодругихявотвседумаючтотудавставитькинокантенырадиоприемни